In [1]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error, r2_score

# 設定超參數
num_classes = 4
num_epochs = 20
batch_size = 8
learning_rate = 0.001
input_size = (640, 640)

# 定義圖像數據集
class BreastDensityDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for density in range(1, num_classes + 1):
            class_dir = os.path.join(self.root_dir, f"breast_density{density}")
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.samples.append((img_path, density - 1))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, target = self.samples[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, target

# 定義圖像轉換
transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 創建數據集和數據加載器
train_dataset = BreastDensityDataset("/home/kevinluo/breast_density_classification/datasets/train", transform=transform)
valid_dataset = BreastDensityDataset("/home/kevinluo/breast_density_classification/datasets/valid", transform=transform)
test_dataset = BreastDensityDataset("/home/kevinluo/breast_density_classification/datasets/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定義模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 20 * 20, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 1)
            )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
  


/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    total = 0
    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        total += labels.size(0)

    return running_loss / total



In [3]:
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs.view(-1), labels.float())

            running_loss += loss.item() * images.size(0)
            total += labels.size(0)

        return running_loss / total


In [4]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [5]:
model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
for epoch in range(num_epochs):
    start_time = time.time()
    train_loss = train(model, train_loader, criterion, optimizer, device)
    valid_loss = validate(model, valid_loader, criterion, device)

    epoch_time = time.time() - start_time
    print(f"Epoch {epoch + 1}/{num_epochs}, Time: {epoch_time:.2f}s, Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}")
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.view(-1).cpu().numpy())


Validation: 100%|██████████| 54/54 [00:06<00:00,  8.75it/s]


Epoch 1/20, Time: 151.29s, Train Loss: 109.7192, Validation Loss: 1.0358


Validation: 100%|██████████| 54/54 [00:06<00:00,  8.26it/s]


Epoch 2/20, Time: 156.07s, Train Loss: 0.7418, Validation Loss: 1.1123


Validation: 100%|██████████| 54/54 [00:06<00:00,  7.82it/s]


...

Validation: 100%|██████████| 54/54 [00:06<00:00,  8.06it/s]


Epoch 19/20, Time: 162.55s, Train Loss: 0.1236, Validation Loss: 0.7262


Validation: 100%|██████████| 54/54 [00:06<00:00,  7.81it/s]


Epoch 20/20, Time: 162.99s, Train Loss: 0.1211, Validation Loss: 0.7280


Testing: 100%|██████████| 72/72 [00:09<00:00,  7.72it/s]


In [12]:
mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
print(f"MSE: {mse:.4f}, R2 Score: {r2:.4f}")

MSE: 0.5201, R2 Score: 0.3341


## mlflow 開始

In [6]:
import mlflow
from datetime import datetime
from torchvision.models import resnet18
import numpy as np

In [7]:
mlflow.set_tracking_uri("http://localhost:5000/")

In [8]:
EXPERIMENT_NAME = "class_to_regression_test"

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME)  # check if the experiment is already exist
if not experiment_id:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment_id.experiment_id

In [9]:
model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
mlflow.start_run(
    experiment_id=experiment_id,
    run_name=f'test_{datetime.now().strftime("%Y-%m-%d")}',
    tags={
        "type": "test",
        "task": "mammogram"
    }
)

<ActiveRun: >

In [11]:
epochs = 20
batch_size = 8
lr = 0.001
mlflow.log_params(
    {
        "epochs": epochs,
        "batch_size": batch_size,
        "lr": lr,
        "num_classes": num_classes,
        "input_size": input_size
    }
)

### training + 畫出process image

In [13]:
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

for epoch in range(num_epochs):
    start_time = time.time()
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
    valid_loss, valid_accuracy = validate(model, valid_loader, criterion, device)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    train_accuracies.append(train_accuracy)
    valid_accuracies.append(valid_accuracy)

    epoch_time = time.time() - start_time
    print(f"Epoch: {epoch+1}/{num_epochs}, Time: {epoch_time:.2f}s, "
        f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
        f"Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

Training: 100%|██████████| 440/440 [02:25<00:00,  3.02it/s]


TypeError: cannot unpack non-iterable float object

In [14]:
for epoch in range(num_epochs):
    start_time = time.time()
    train_loss = train(model, train_loader, criterion, optimizer, device)
    valid_loss = validate(model, valid_loader, criterion, device)

    epoch_time = time.time() - start_time
    print(f"Epoch {epoch + 1}/{num_epochs}, Time: {epoch_time:.2f}s, Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}")
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(outputs.view(-1).cpu().numpy())

Validation: 100%|██████████| 54/54 [00:06<00:00,  7.93it/s]


Epoch 1/20, Time: 155.82s, Train Loss: 0.7445, Validation Loss: 0.9205


Validation: 100%|██████████| 54/54 [00:06<00:00,  8.15it/s]


Epoch 2/20, Time: 160.98s, Train Loss: 0.7000, Validation Loss: 1.0533


Validation: 100%|██████████| 54/54 [00:06<00:00,  7.78it/s]


...

Validation: 100%|██████████| 54/54 [00:06<00:00,  7.96it/s]


Epoch 19/20, Time: 163.47s, Train Loss: 0.1432, Validation Loss: 0.7634


Validation: 100%|██████████| 54/54 [00:06<00:00,  7.88it/s]


Epoch 20/20, Time: 162.77s, Train Loss: 0.1337, Validation Loss: 0.7399


Testing: 100%|██████████| 72/72 [00:08<00:00,  8.04it/s]


In [15]:
mlflow.end_run()